<a href="https://colab.research.google.com/github/Tuhinm2002/kaggle_competitions/blob/main/classification/binary_prediction_of_poisonous_mushrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
! kaggle competitions download -c playground-series-s4e8

 89% 73.0M/82.3M [00:00<00:00, 117MB/s]
100% 82.3M/82.3M [00:00<00:00, 113MB/s]


In [ ]:
! unzip playground-series-s4e8

Archive:  playground-series-s4e8.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [45]:
sample_data = pd.read_csv("sample_submission.csv")
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import SGD,AdamW
from tensorflow.keras.layers import Layer,Dense,BatchNormalization,Input,Flatten
from tensorflow.keras.losses import Loss,MSE,BinaryCrossentropy


from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef

In [ ]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [ ]:
test_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,17.12,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,10.75,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,3.14,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,8.51,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,13.70,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u


In [ ]:
train_data.isna().sum()

,0
id,0
class,0
cap-diameter,4
cap-shape,40
cap-surface,671023
cap-color,12
does-bruise-or-bleed,8
gill-attachment,523936
gill-spacing,1258435
gill-color,57


In [ ]:
len(train_data) - 128880

2988065

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit_transform(train_data['ring-type'])

array([18, 39, 18, ..., 39, 27, 18])

In [46]:
def encode_val(val):
    try :
        return float(val)
    except :
        return val

for x_col1 in train_data.drop(columns=['id']).columns:
    train_data[x_col1] = train_data[x_col1].apply(lambda x: encode_val(x))

for x_col2 in test_data.drop(columns=['id']).columns:
    test_data[x_col2] = test_data[x_col2].apply(lambda x: encode_val(x))

In [ ]:
train_data.columns

Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [ ]:
print(train_data['cap-shape'].mode())
print(train_data['cap-shape'].mode()[0])

0    x
Name: cap-shape, dtype: object
x


In [47]:
def fix_missing(data,col):
    if data[col].dtypes == 'float64' or data[col].dtypes == 'int64':
        return data[col].fillna(data[col].mean(),inplace=True)
    else:
        return data[col].fillna(data[col].mode()[0],inplace=True)

for x_col1 in train_data.drop(columns=['id','class']).columns:
    fix_missing(train_data,x_col1)

for x_col2 in test_data.drop(columns=['id']).columns:
    fix_missing(test_data,x_col2)

In [ ]:
train_data.isna().sum()

,0
id,0
class,0
cap-diameter,0
cap-shape,0
cap-surface,0
cap-color,0
does-bruise-or-bleed,0
gill-attachment,0
gill-spacing,0
gill-color,0


In [ ]:
print(train_data['cap-diameter'].dtypes == 'float64')

True


In [ ]:
train_data.loc[0,'cap-diameter']

8.8

In [ ]:
# for j in range(len(train_data['veil-color'].values)):
#     if isinstance(train_data['veil-color'].values[j],int) or isinstance(train_data['veil-color'].values[j],float):
#         print(train_data['veil-color'].values[j])


# def value_exchange():
#     arr = train_data['veil-color'].values
#     with np.nditer(arr, flags=['refs_ok']) as it:
#         for n1 in it:
#             if isinstance(n1.item(), int) or isinstance(n1.item(), float):
#                 print(n1.item())
# value_exchange()

# with np.nditer(train_data['veil-color'].values, flags=['refs_ok']) as it:
#     for n1 in it:
#         if isinstance(n1.item(), int) or isinstance(n1.item(), float):
#             print(n1.item())

In [48]:
def label_encode(data,col):
    vals = list(data[col].unique())
    arr = data[col].values
    k = 0.0
    d = {}
    for i in range(len(vals)):
        if not isinstance(vals[i],int) and not isinstance(vals[i],float):
            d[vals[i]] = k
            k+=1

    with np.nditer(arr, flags=['refs_ok']) as it:
        for j,n1 in enumerate(it):
            if not isinstance(n1.item(), int) and not isinstance(n1.item(), float):
                arr[j] = float(d[n1.item()])
            else:
                continue
    data[col] = pd.DataFrame(arr,columns=[col],dtype=np.float64)
    return data[col]

for x_col1 in train_data.drop(columns=['id','cap-diameter']).columns:
    train_data[x_col1] = label_encode(train_data,x_col1)

In [49]:
for x_col2 in test_data.drop(columns=['id','cap-diameter']).columns:
    label_encode(test_data,x_col2)

In [ ]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,0.0,8.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.51,15.39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.0,4.51,1.0,1.0,1.0,0.0,0.0,0.0,1.0,4.79,6.48,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,2,0.0,6.94,0.0,0.0,2.0,0.0,1.0,0.0,0.0,6.85,9.93,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,3,0.0,3.88,0.0,2.0,3.0,0.0,2.0,0.0,2.0,4.16,6.53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,4,0.0,5.85,1.0,3.0,4.0,0.0,3.0,0.0,0.0,3.37,8.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [ ]:
train_data['cap-shape']

,cap-shape
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
3116940,0.0
3116941,6.0
3116942,1.0
3116943,2.0


In [ ]:
type(train_data.loc[0,'cap-shape'])

numpy.float64

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(train_data.drop(columns=['id','class']),train_data['class'],test_size=0.3,random_state=0)

In [ ]:
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()

In [ ]:
Y_test

array([1., 0., 0., ..., 1., 1., 0.])

In [ ]:
train_data.shape

(3116945, 22)

In [ ]:
X_train.shape

(2181861, 20)

In [ ]:
# from tensorflow.keras import backend as k
# from keras.src import ops
# from keras.src.losses.loss import squeeze_or_expand_to_same_rank



class MushroomClassifier(Model):
    def __init__(self):
        super().__init__()

        self.layer1 = Sequential([
                        Input(shape=(20,),dtype=np.float64),
                        Dense(128,activation='relu'),
                        Dense(64,activation='relu'),
                        Dense(32,activation='relu'),
                        Dense(16,activation='relu'),
                        Flatten(),
                        Dense(1,activation='sigmoid')])

    def call(self,x,training):
        return self.layer1(x)



m_model = MushroomClassifier()
print(m_model.summary())

Model: "mushroom_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 1)                   │          13,569 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,569 (53.00 KB)

 Trainable params: 13,569 (53.00 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
m_model.compile(loss=BinaryCrossentropy,optimizer=SGD(learning_rate=0.01),metrics=["acc"])

In [ ]:
def MCCLoss(y_true,y_pred):

  y_pred = tf.round(tf.clip_by_value(y_pred, 0, 1))
  return tf.constant(matthews_corrcoef(tf.constant(y_true),tf.round(tf.constant(y_pred))))



In [ ]:
tf.config.run_functions_eagerly(True)

In [38]:
m_model.fit(X_train,Y_train,verbose=1,epochs=2)

Epoch 1/2
68184/68184 ━━━━━━━━━━━━━━━━━━━━ 2989s 44ms/step - acc: 0.9461 - loss: 0.1529
Epoch 2/2
68184/68184 ━━━━━━━━━━━━━━━━━━━━ 3163s 46ms/step - acc: 0.9835 - loss: 0.0644
68184/68184 ━━━━━━━━━━━━━━━━━━━━ 3163s 46ms/step - acc: 0.9835 - loss: 0.0644


In [50]:
preds = tf.round(m_model.predict(test_data.drop(columns=['id'])))

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


   14/64937 ━━━━━━━━━━━━━━━━━━━━ 8:52 8ms/step

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


64937/64937 ━━━━━━━━━━━━━━━━━━━━ 758s 12ms/step


In [ ]:
np.round([[0.55788],[0.45]])

In [ ]:
from sklearn.metrics import matthews_corrcoef

y_true = [+1, +1, +1, -1]

y_pred = [+1, -1, +1, +1]

matthews_corrcoef(y_true, y_pred)

In [ ]:
# l = MCCLoss()
print(MCCLoss(tf.constant([+1, +1, +1, -1]),tf.constant([+1, -1, +1, +1])))

In [ ]:
tf.round(tf.clip_by_value(tf.constant([+1, +1, +1, -1]), 0, 1)).numpy()

In [51]:
preds

<tf.Tensor: shape=(2077964, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)>

In [57]:
preds = preds.numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [58]:
preds = preds.astype('object')

In [59]:
with np.nditer(preds, flags=['refs_ok']) as it:
    for j,n1 in enumerate(it):
      if n1.item() == 0.0:
        preds[j] = 'e'
      elif n1.item() == 1.0:
        preds[j] = 'p'

In [60]:
final_output = pd.concat([pd.DataFrame(test_data['id']),pd.DataFrame(preds)],axis=1,ignore_index=True)

In [61]:
final_output

,0,1
0,3116945,p
1,3116946,p
2,3116947,p
3,3116948,e
4,3116949,e
...,...,...
2077959,5194904,e
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e


In [62]:
final_output.rename(columns={0:'id',1:'class'},inplace=True)

In [63]:
final_output.to_csv("submission.csv",index=False)